# Lesson 7 — *Project*

**Goal:** Extend your Lesson 5/6 profile with **savings** and a **zakat calculator**.
- Store your savings in the API.
- Add money with a button (POST).
- Calculate zakat (2.5%) with one click.

Keep everything on the **same domain** (Flask + HTML + CSS together), so no CORS issues.

## 1) Profile + Savings (Starter)
We'll reuse the Lesson 5/6 idea but add **savings** and **zakat** (2.5%).
- Keep everything simple and on the same domain.
- You can change the profile fields to be about you.

In [ ]:
# app.py (starter code) — keeps everything in one folder (app.py + index.html)
from flask import Flask, jsonify, request

app = Flask(__name__, static_folder='.', static_url_path='')

# Your profile with savings
profile = {
    "name": "Fatimah",
    "city": "Dallas",
    "school": "BHA",
    "favorite_color": "blue",
    "hobbies": ["drawing", "coding", "football"],
    "savings": 120.0,  # dollars
}

ZAKAT_RATE = 0.025  # 2.5%

@app.route("/")
def home():
    return app.send_static_file("index.html")

@app.route("/profile")
def get_profile():
    return jsonify(profile)

@app.route("/add-saving", methods=["POST"])
def add_saving():
    data = request.get_json(silent=True) or {}
    try:
        amount = float(data.get("amount", 0))
    except Exception:
        return jsonify({"error": "Amount must be a number"}), 400
    if amount <= 0:
        return jsonify({"error": "Amount must be positive"}), 400
    profile["savings"] += amount
    return jsonify({"savings": round(profile["savings"], 2)})

@app.route("/zakat")
def calc_zakat():
    zakat = profile["savings"] * ZAKAT_RATE
    return jsonify({"zakat": round(zakat, 2), "rate": ZAKAT_RATE})

if __name__ == "__main__":
    app.run(debug=True)


## 2) Build the page (writes `index.html`)
This page will:
- Load your profile from `/profile` (name, city, school, hobbies, favorite color, savings).
- Let you **add to savings** with a number input + button (POST `/add-saving`).
- Show **zakat (2.5%)** when you click a button (GET `/zakat`).
- Everything runs on the **same domain** as your Flask app (no CORS).

In [ ]:
%%html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1.0" />
  <title>Profile + Savings + Zakat</title>
  <link rel="stylesheet" href="/style.css" />
</head>
<body>
  <div class="card">
    <div class="top">
      <div class="avatar" id="avatar">🙂</div>
      <div>
        <h1 id="name">Loading...</h1>
        <p class="muted" id="city">City: ...</p>
        <p class="muted" id="school">School: ...</p>
        <div class="pill" id="colorText">Favorite color: ...</div>
      </div>
    </div>

    <div class="grid">
      <div class="info-box">
        <div class="label">Savings</div>
        <div class="value" id="savings">—</div>
      </div>
      <div class="info-box">
        <div class="label">Zakat (2.5%)</div>
        <div class="value" id="zakat">Tap calculate</div>
      </div>
      <div class="info-box">
        <div class="label">Favorite Color</div>
        <div class="value color-row"><span class="color-dot" id="colorDot"></span><span id="colorValue">—</span></div>
      </div>
    </div>

    <div class="section">
      <h3>Hobbies</h3>
      <div class="tags" id="hobbyTags"></div>
    </div>

    <div class="section">
      <h3>Add to Savings</h3>
      <div class="row">
        <input id="amountInput" type="number" min="1" step="1" placeholder="e.g., 10" />
        <button id="btnAdd">➕ Add</button>
      </div>
      <p id="statusMsg" class="muted"></p>
    </div>

    <div class="section">
      <h3>Calculate Zakat</h3>
      <button id="btnZakat">🧮 Calculate</button>
    </div>
  </div>

  <script>
    const statusMsg = document.getElementById('statusMsg');

    function firstLetter(text) {
      return (text && text.trim()) ? text.trim()[0].toUpperCase() : '🙂';
    }

    async function loadProfile() {
      const r = await fetch('/profile');
      const data = await r.json();
      const name = data.name || 'Unknown';
      document.getElementById('name').textContent = name;
      document.getElementById('avatar').textContent = firstLetter(name);

      document.getElementById('city').textContent = 'City: ' + (data.city || '—');
      document.getElementById('school').textContent = 'School: ' + (data.school || '—');
      document.getElementById('colorText').textContent = 'Favorite color: ' + (data.favorite_color || '—');
      document.getElementById('colorValue').textContent = data.favorite_color || '—';
      if (data.favorite_color) document.getElementById('colorDot').style.background = data.favorite_color;

      document.getElementById('savings').textContent = '$' + (data.savings ?? 0);

      const tags = document.getElementById('hobbyTags');
      tags.innerHTML = '';
      (data.hobbies || []).forEach(h => {
        const span = document.createElement('span');
        span.className = 'tag';
        span.textContent = h;
        tags.appendChild(span);
      });
    }

    async function addSaving() {
      statusMsg.textContent = 'Saving...';
      const amtInput = document.getElementById('amountInput');
      const amt = Number(amtInput.value || 0);
      const r = await fetch('/add-saving', {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify({ amount: amt })
      });
      const data = await r.json();
      if (!r.ok) {
        statusMsg.textContent = data.error || 'Error';
        statusMsg.className = 'error';
        return;
      }
      document.getElementById('savings').textContent = '$' + data.savings;
      statusMsg.textContent = '✅ Added!';
      statusMsg.className = 'ok';
      amtInput.value = '';
    }

    async function calcZakat() {
      document.getElementById('zakat').textContent = '...';
      const r = await fetch('/zakat');
      const data = await r.json();
      document.getElementById('zakat').textContent = '$' + (data.zakat ?? 0);
    }

    document.getElementById('btnAdd').addEventListener('click', addSaving);
    document.getElementById('btnZakat').addEventListener('click', calcZakat);

    loadProfile();
  </script>
</body>
</html>

## 2) Build the page (index.html)
- Show profile info (name, city, school, savings, hobbies, favorite color).
- Input + button: **Add to Savings** (POST `/add-saving`).
- Button: **Calculate Zakat** (GET `/zakat`) and show the amount.
- Keep it simple and friendly for kids.

## 3) Your Turn (add one simple feature)
Choose any one:
1) **Saving goal**: Add `"saving_goal"` to the profile and show it under savings.
2) **Reset savings**: Make a new `/reset-savings` POST endpoint that sets savings to 0 and add a button for it.
3) **Colorful zakat**: When zakat is calculated, change the zakat text color to green.

Keep it tiny—just one of these is enough!